In order to find the implied volatility $\sigma_{implied}$, one needs to find the inverse of the Black-Scholes Function wiith respect to $\sigma_{implied}$. Unfortunately, no closed-form inverse exists so we resort to numerical methods such as the Newton-Raphson Method. The implied volatility can be calculated using the following equation:

$$BS(\sigma_{implied}, r, T, K, S_0) - V_{call}^{market} )$$

Under the assumption that $g(x) = BS(x) - V_{call}^{market}$ is differentiable, define $\epsilon_x = x_{ex} - x_n$ and perform a taylor series expansion such that 
$$0 = g(x_{ex}) = g(x_n) + \epsilon_n g'(x_n) + \frac{\epsilon_n^2}{2} g''(x_n)$$

Plugging in the definition of $\epsilon$ results in:
$$x_{n+1} = x_{n} - \frac{g(x_n)}{g'(x_n)}$$

Note that $BS(x) = \sigma_{implied}$, then the formula also has the following representation:
$$\sigma_{n+1} = \sigma_n - \frac{BS(\sigma_n, \cdot) - V_{call}^{market}}{ \frac{\partial BS(\sigma_n, \cdot)}{\partial \sigma_n} }$$

Note that $\frac{\partial BS(\sigma_n, \cdot)}{\partial \sigma_n}$ is also known as vega and is known analytically

In [1]:
import numpy as np
import scipy.stats as st
# Initial parameters and market quotes
V_market = 2    # market call option price
K        = 120  # strike
tau      = 1    # time-to-maturity
r        = 0.05 # interest rate
S_0      = 100  # today's stock price
sigmaInit    = 0.25  # Initial implied volatility
CP       ="c" #C is call and P is put

# Vega, dV/dsigma
def dV_dsigma(S_0,K,sigma,tau,r):
    #parameters and value of Vega
    d2   = (np.log(S_0 / float(K)) + (r - 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    value = K * np.exp(-r * tau) * st.norm.pdf(d2) * np.sqrt(tau)
    return value

def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    #Black-Scholes Call option price
    d1    = (np.log(S_0 / float(K)) + (r + 0.5 * np.power(sigma,2.0)) * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if str(CP).lower()=="c" or str(CP).lower()=="1":
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif str(CP).lower()=="p" or str(CP).lower()=="-1":
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value

def ImpliedVolatility(CP,S_0,K,sigma,tau,r):
    error    = 1e10; # initial error
    optPrice = lambda sigma: BS_Call_Option_Price(CP,S_0,K,sigma,tau,r)
    vega = lambda sigma: dV_dsigma(S_0,K,sigma,tau,r)
    
    # While the difference between the model and the market price is large
    # follow the iteration
    n = 1.0 
    while error>10e-10:
        g         = optPrice(sigma) - V_market
        g_prim    = vega(sigma)
        sigma_new = sigma - g / g_prim
    
        #error=abs(sigma_new-sigma)
        error=abs(g)
        sigma=sigma_new;
        
        print('iteration {0} with error = {1}'.format(n,error))
        
        n= n+1
    return sigma

sigma_imp = ImpliedVolatility(CP,S_0,K,sigmaInit,tau,r)
message = '''Implied volatility for CallPrice= {}, strike K={}, 
      maturity T= {}, interest rate r= {} and initial stock S_0={} 
      equals to sigma_imp = {:.7f}'''.format(V_market,K,tau,r,S_0,sigma_imp)
            
print(message)

val = BS_Call_Option_Price(CP,S_0,K,sigma_imp,tau,r)

# Verify the result for the option price is consistent with the assumption values
print('Option Price for implied volatility of {0} is equal to {1}'.format(sigma_imp, val))

iteration 1.0 with error = 3.025413481792615
iteration 2.0 with error = 0.19134998568795325
iteration 3.0 with error = 0.0022254541477302325
iteration 4.0 with error = 3.353154056640051e-07
iteration 5.0 with error = 1.0658141036401503e-14
Implied volatility for CallPrice= 2, strike K=120, 
      maturity T= 1, interest rate r= 0.05 and initial stock S_0=100 
      equals to sigma_imp = 0.1614827
Option Price for implied volatility of 0.1614827288413938 is equal to 2.0
